### Helpers

In [42]:
from pprint import pprint

def get_physical_signal_name(setup, quid, signal_name):
    logical_signal = setup.logical_signal_groups[quid].logical_signals[signal_name]
    return logical_signal.physical_channel.uid


def print_qpu_signals(setup):
    qubit_signals = {
        quid: list(lsg.logical_signals) for quid, lsg in setup.logical_signal_groups.items()
    }
    connections = {
        quid: {sig_name: get_physical_signal_name(setup, quid, sig_name) for sig_name in signals}
        for quid, signals in qubit_signals.items()
    }


    pprint(connections)

### Simple setup

In [43]:
# Setting get_zsync=True automatically detects the zsync ports of the PQCS that
# are used by the other instruments in this descriptor.

from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
from laboneq.simple import DeviceSetup

descriptor = generate_descriptor(
    pqsc=["DEV10001"],
    hdawg_8=["DEV8001"],
    shfqc_6=["DEV12001"],
    number_data_qubits=2,
    number_flux_lines=0,
    include_cr_lines=False,
    multiplex=True,
    number_multiplex=2,
    get_zsync=False,
    ip_address="localhost",
)
setup = DeviceSetup.from_descriptor(descriptor, "localhost")

print_qpu_signals(setup)

{'q0': {'acquire': 'SHFQC_DEV12001/qachannels_0_input',
        'drive': 'SHFQC_DEV12001/sgchannels_0_output',
        'drive_ef': 'SHFQC_DEV12001/sgchannels_0_output',
        'measure': 'SHFQC_DEV12001/qachannels_0_output'},
 'q1': {'acquire': 'SHFQC_DEV12001/qachannels_0_input',
        'drive': 'SHFQC_DEV12001/sgchannels_1_output',
        'drive_ef': 'SHFQC_DEV12001/sgchannels_1_output',
        'measure': 'SHFQC_DEV12001/qachannels_0_output'}}


### Advanced setup

In [44]:
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup,
)

# specify the number of qubits you want to use
number_of_qubits = 2

# generate the device setup using a helper function
setup = generate_device_setup(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[
        {
            "serial": "DEV8001",
            "zsync": 0,
            "number_of_channels": 8,
            "options": "HDAWG8/CNT/MF/ME",
        }
    ],
    shfqc=[
        {
            "serial": "DEV12001",
            "zsync": 1,
            "number_of_channels": 2*number_of_qubits,
            "readout_multiplex": number_of_qubits,
            "options": "SHFQC/QC6CH/PLUS/RTR",
        }
    ],
    include_flux_lines=False,
    multiplex_drive_lines=True,  # adds drive_ef
    server_host="localhost",
    setup_name="my_setup",
)


# Print signals
print_qpu_signals(setup)

{'q0': {'acquire': 'shfqc_0/qachannels_0_input',
        'drive': 'shfqc_0/sgchannels_0_output',
        'drive_ef': 'shfqc_0/sgchannels_0_output',
        'measure': 'shfqc_0/qachannels_0_output'},
 'q1': {'acquire': 'shfqc_0/qachannels_0_input',
        'drive': 'shfqc_0/sgchannels_1_output',
        'drive_ef': 'shfqc_0/sgchannels_1_output',
        'measure': 'shfqc_0/qachannels_0_output'}}
